In [111]:
from ipyleaflet import (Map, GeoData, basemaps, WidgetControl, GeoJSON, LayersControl,
                        Choropleth, Heatmap, SearchControl,FullScreenControl, LegendControl)
from ipywidgets import Text, HTML, widgets, interact
from branca.colormap import linear
import geopandas as gpd
import pandas as pd
import json

from datenguidepy import get_regions, get_statistics, Query

# data from datenguide API

Hesse has region code '06'

In [4]:
get_regions().query("level == 'nuts1'")

,name,level,parent
region_id,,,
10,Saarland,nuts1,DG
11,Berlin,nuts1,DG
12,Brandenburg,nuts1,DG
13,Mecklenburg-Vorpommern,nuts1,DG
14,Sachsen,nuts1,DG
15,Sachsen-Anhalt,nuts1,DG
16,Thüringen,nuts1,DG
01,Schleswig-Holstein,nuts1,DG
02,Hamburg,nuts1,DG


In [69]:
selected_stats = 'AI1903'
year = 2013

In [91]:
def get_data_to_show(selected_stats=selected_stats, year = year):
    q = Query.all_regions(parent='06')
    stat = q.add_field(selected_stats)
    stat.add_args({'year': [year]})

    data = q.results(verbose_enums=True)[['name', selected_stats]]
    # remove Kreis & kreisfreie Stadt from names to be compatible with geojson ids
    data['name'] = data['name'].apply(lambda x: x.split(',')[0])

    data = dict(data.to_dict('split')['data'])
    return data 

# get geoJSON

In [37]:
geojson_data = json.load(open('../data/landkreise_simplify200.geojson','r'))

In [38]:
pop = dict()
for feature in geojson_data['features']:
    pop[feature['properties']['GEN']] = feature['properties']['destatis']['population']    
pop

{'Darmstadt': 159207,
 'Frankfurt am Main': 753056,
 'Offenbach am Main': 128744,
 'Wiesbaden': 278342,
 'Bergstraße': 269694,
 'Darmstadt-Dieburg': 297399,
 'Groß-Gerau': 274526,
 'Hochtaunuskreis': 236564,
 'Main-Kinzig-Kreis': 418950,
 'Main-Taunus-Kreis': 237735,
 'Odenwaldkreis': 96798,
 'Offenbach': 354092,
 'Rheingau-Taunus-Kreis': 187157,
 'Wetteraukreis': 306460,
 'Gießen': 268876,
 'Lahn-Dill-Kreis': 253777,
 'Limburg-Weilburg': 172083,
 'Marburg-Biedenkopf': 246648,
 'Vogelsbergkreis': 105878,
 'Kassel': 236633,
 'Fulda': 222584,
 'Hersfeld-Rotenburg': 120829,
 'Schwalm-Eder-Kreis': 180222,
 'Waldeck-Frankenberg': 156953,
 'Werra-Meißner-Kreis': 101017}

In [54]:
geo_data = {'features':[], 'id':[]}
for f in geojson_data['features']:
    f.update(id=f['properties']['GEN'])
    geo_data['features'].append(f)
#geo_data

# plot data on map

In [192]:
stats_widget = widgets.Dropdown(
    options=['AI1903', 'AI1904'],
    value='AI1903',
    description='Statistics:',
    disabled=False,
)

year_widget = widgets.Dropdown(
    options= range(2010,2019),
    value=2018,
    description='year:',
    disabled=False,
)

In [189]:

def show_map(selected_stats, year):
     choro_data = get_data_to_show(selected_stats, year),

     layer = Choropleth(
          geo_data= geo_data,
          choro_data= choro_data[0],
     #     key_on = 'features',
          colormap=linear.YlOrRd_04,
          style={'fillOpacity': 0.8, 'dashArray': '5, 5'}
     )

     m = Map(center=(50.5,9), zoom=8)
     m.add_layer(layer)
     m.layout.width = '50%'
     m.layout.height = '700px'

     min_value, max_value = min(choro_data[0].values()), max(choro_data[0].values())
     # custom made "color bar"
     legend = LegendControl(
          {str(min_value): linear.YlOrRd_04.rgb_hex_str(0),
          str(min_value+0.5*(max_value-min_value)): linear.YlOrRd_04.rgb_hex_str(.5),
          str(max_value): linear.YlOrRd_04.rgb_hex_str(1)},
          name=selected_stats, position="bottomright")
     m.add_control(legend)
     return m

In [191]:
interact(show_map, selected_stats=stats_widget, year=year_widget);

interactive(children=(Dropdown(description='Statistics:', index=1, options=('AI1903', 'AI1904'), value='AI1904…

In [33]:
geojson_data

UsageError: Line magic function `%skip` not found.


In [34]:
geo_data